In [2]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
init_printing(use_latex='mathjax')

# Asymptotic Expansions

## Order notation

Order notation is a way to compare the growth of one function to another in a particular limit $x\to a$. The most famous example is Big O notation, where we say that $f(x) = O(g(x))$ as $x\to a$ if $$\limsup_{x\to a}\frac{|f(x)|}{|g(x)|} < \infty.$$ The other term utilized in asymptotics is little o, where $f(x) = o(g(x))$ as $x\to a$ if $$\limsup_{x\to a}\frac{|f(x)|}{|g(x)|} = 0, g\neq0.$$

Other symbols are also commonly used to express these ideas, so we might say $f \ll g$ if $f=o(g)$ or $f\sim g$ if $f/g\to1$. Computer scientists and programmers might also be familiar with big and little $\omega$ s and $\theta$ s, but that specificity is outside the scope of this work. And while the computer scientist often uses these notations in the limit as $n\to\infty$ in the context of algorithmic complexity and space, in asymptotics limits can be for variables like $x$ or parameters like $\epsilon$, and often at limits of $0$ as well as $\infty.$

Sympy is nice in that it can handle big O notation fairly well, even with multiple variables, defaulting to a limit of 0. From their documentation

In [4]:
x,y = symbols('x y')
display(O(1 + x*y))
display(O(1 + x*y, (x, oo), (y, oo))) # oo is infty
display(O(1 + x*y, (x, 0), (y, 0)))

O(1; (x, y) → (0, 0))

O(x⋅y; (x, y) → (∞, ∞))

O(1; (x, y) → (0, 0))

## Expansions

We define an asymptotic sequence to be a sequence of gauge functions $\{f_n\}$ such that $f_{n+1}=o(f_n)$ at some limit. An asympotic expansion of $f$ is when we have a sequence of coefficients ${a_n}$ such that $$f(x)-\sum_{n=0}^N\phi_n = o(\phi_N)$$ at the same limit, also denoted $f\sim\sum a_n\phi_n$

For a given sequence of gauge functions, we can compute the coefficients with the following formula: $$a_{N+1} = \lim \frac{f(x) - \sum_{n=0}^N a_n\phi_n(x)}{\phi_{N+1}(x)}$$

The following code shows such an example.

In [11]:
f = log(sin(x))
phi = [log(x), x**2, x**4, x**6]
f_asymp = 0
for p in phi:
    a = limit((f-f_asymp)/p,x,0,'+')
    f_asymp = f_asymp + a*p

display(f,f_asymp)

log(sin(x))

    6     4     2         
   x     x     x          
- ──── - ─── - ── + log(x)
  2835   180   6          

### Operations on Asymptotic Expansions

- Addition and Scaling: No problems
- Multiplication: could get messy with the wrong gauge functions, but nothing in principle
- Term-by-term integration: Valid
- Term-by-term differentiation: could break down with highly oscillatory terms, e.g. $\sin(1/x)$

Broadly speaking, pathologies of a function will not go away with an asymptotic expansion. 

### Asymptotic Power Series

A Taylor series expansion is a valid asymptotic expansion.

[Borel's Lemma](https://en.wikipedia.org/wiki/Borel's_lemma) can be used to show that any asymptotic power series is asymptotic to some $C^\infty$ function.

### Convergent versus Asymptotic Series

Consider the series $S_N(x) = \sum_{n=0}^N a_n\phi_n(x).$ An asymptotic series is concerned with the behavior in some particular limit of $x$, where as convergence is concerned with the behavior in the limit $N\to\infty$.

The following example takes the same function $\text{erf}(x) = \frac{2}{\sqrt{\pi}}\int_0^x e^{-y^2} dy$ and constructs a convergent, asymptotic series as $x\to0$ (the Taylor Series), and a divergent, asymptotic series as $x\to\infty.$

Despite being covergent, the Taylor series does not do a great job at evaluating the function at $x=3$, whereas the four terms of the divergent series yields six digits of accuracy. That is not to say that more terms in the divergent series immediately leads to more accuracy; it tends to perform well up to a certain number of terms before the divergence takes over.

In [42]:
f = 2/sqrt(pi) * integrate(exp(-y**2), (y,0,x))
eval3 = lambda f: N(f.subs({x:3}).simplify())
display(eval3(f))
phi = [x,x**3,x**5,x**7]
f_asymp = 0
for p in phi:
    a = limit((f-f_asymp)/p,x,0,'+')
    f_asymp = f_asymp + a*p
display(f_asymp,eval3(f_asymp))

phi = [1,exp(-x**2)*x**-1,exp(-x**2)*x**-3,exp(-x**2)*x**-5]
f_asymp = 0
for p in phi:
    a = limit((f-f_asymp)/p,x,oo)
    f_asymp = f_asymp + a*p
display(f_asymp,eval3(f_asymp))

0.999977909503001

    7       5       3      
   x       x     2⋅x    2⋅x
- ───── + ──── - ──── + ───
  21⋅√π   5⋅√π   3⋅√π   √π 

-38.1069764430542

       2       2          2 
     -x      -x         -x  
    ℯ       ℯ        3⋅ℯ    
1 - ──── + ─────── - ───────
    √π⋅x         3         5
           2⋅√π⋅x    4⋅√π⋅x 

0.999977865641434

### Stokes Phenomenon

If an asymptotic expansion is taking place over the complex numbers, then as $z$ approaches any essential singularity (often at $\infty$), then the asymptotic expansion will often be depend on which 'slice' of the complex plane it approaches the singularity from. For example, our asymptotic expansion for $\text{erf}$ has a leading coefficient of 1, if we performed it in the limit $x\to-\infty$ the lead coefficient would become -1 (you can try this), or zero as $x\to \pm i\infty$ (sympy does not support this). 